In [0]:
def lstm_forecast(lag, model_name, testing_set_csv):
    #--------------------------------------------------------------------------------------------------------------------
    # Test the model on a csv input file
    #--------------------------------------------------------------------------------------------------------------------
    #-----------------------------------------------IMPORT TIME & DATETIME-----------------------------------------------
    import time
    import datetime
    #-----------------------------------------------IMPORT TIME & DATETIME-----------------------------------------------
    start_time = time.time()

    #-----------------------------------------------IMPORT PANDAS--------------------------------------------------------
    import pandas as pd
    #-----------------------------------------------IMPORT PANDAS--------------------------------------------------------

    #--------------------------------------------------------------------------------------------------------------------
    #Inputs:
    #latest_model = "20181001161933.h5"
    #"test_set_size=369_week=7"
    #"test_set_size=312_week=8"
    #"test_set_size=347_week=9"
    #testing_set_csv = "test_set_size=1219_week=18.csv"

    #Output:
    #testing_set_out = "testset_size=1219_week=18"
    #--------------------------------------------------------------------------------------------------------------------



    #from allPaths import all_paths
    data_and_scripts_location = all_paths()
    models_path = data_and_scripts_location["Models"]
    testing_data_path = data_and_scripts_location["Data/parsed"]
    forecast_output_path = data_and_scripts_location["Forecasts"]

    dataset_test = pd.read_csv(testing_data_path + "/" + testing_set_csv)

    dataset_test['Partition'] = dataset_test['Partition'].astype('category')
    dataset_test['Partition'] = dataset_test['Partition'].cat.codes

    necessaryFeatures=["Partition","ReqNodes","ReqCPUS","NNodes","Timelimit","sin(Weekday)","cos(Weekday)","sin(Dayminute)","cos(Dayminute)","QueueTime"]
    test_set = dataset_test[necessaryFeatures].values

    #test_qt_list = dataset_test["QueueTime"].values
    test_qt = dataset_test.iloc[:, 11].values
    test_qt_list = test_qt.tolist()

    #-----------------------------------------------IMPORT NUMPY--------------------------------------------------------
    import numpy as np
    #-----------------------------------------------IMPORT NUMPY--------------------------------------------------------
    # Compute size of the dataset
    testsetHeight, testsetWidth = np.shape(test_set)

    #-----------------------------------------------IMPORT OneHotEncoder------------------------------------------------
    from sklearn.preprocessing import OneHotEncoder
    #-----------------------------------------------SKLEARN-------------------------------------------------------------

    onehotencoder_1 = OneHotEncoder(categorical_features = [0])
    test_set = onehotencoder_1.fit_transform(test_set).toarray()


    ####################################################################################################################
    # Avoid dummy variable trap
    #test_set_ind = np.delete(test_set, 3, 1)
    ####################################################################################################################

    numberIndFeatures = 13

    test_set_new = np.zeros((testsetHeight,numberIndFeatures))
    test_set_new[:, 0:numberIndFeatures-1] = test_set[:, 0:numberIndFeatures-1]

    queueTimes = test_set[:, numberIndFeatures-1]
    i = -1
    for q in queueTimes:
        i += 1
        if q == 0:
            test_set_new[i, numberIndFeatures-1] = 1 # first class
        elif q < 5:
            test_set_new[i, numberIndFeatures-1] = 2 # second class
        elif q < 30:
            test_set_new[i, numberIndFeatures-1] = 3 # third class
        elif q < 60:
            test_set_new[i, numberIndFeatures-1] = 4 # fourth class
        elif q < 120:
            test_set_new[i, numberIndFeatures-1] = 5 # fifth class
        elif q < 240:
            test_set_new[i, numberIndFeatures-1] = 6 # sixth class
        elif q < 600:
            test_set_new[i, numberIndFeatures-1] = 7 # seventh class
        elif q > 600:
            test_set_new[i, numberIndFeatures-1] = 8 # eighth class

    #-----------------------------------------------IMPORT MinMaxScaler-------------------------------------------------
    from sklearn.preprocessing import MinMaxScaler
    #-----------------------------------------------SKLEARN-------------------------------------------------------------

    sc = MinMaxScaler(feature_range = (0, 1))
    test_set_scaled = sc.fit_transform(test_set_new[:, 0:numberIndFeatures-1])

    #-----------------------------------------------IMPORT LabelEncoder-------------------------------------------------
    from sklearn.preprocessing import LabelEncoder
    #-----------------------------------------------SKLEARN-------------------------------------------------------------

    test_set_new_class = test_set_new[:, numberIndFeatures-1]
    test_set_new_class_reshape = test_set_new_class.reshape(test_set_new_class.size, 1)
    test_set_scaled = np.append(test_set_scaled, test_set_new_class_reshape, axis = 1)
    labelencoder = LabelEncoder()
    test_set_scaled[:, numberIndFeatures-1] = labelencoder.fit_transform(test_set_scaled[:, numberIndFeatures-1])

    X_test = []
    lag = 50
    for i in range(lag, test_set_scaled[:,0].size):
        X_test.append(test_set_scaled[i-lag:i, 0:numberIndFeatures-1])  
    X_test = np.array(X_test)

    # Reshaping training set
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], numberIndFeatures-1))



    #-----------------------------------------------IMPORT KERAS-------------------------------------------------
    from keras.models import load_model
    #-----------------------------------------------IMPORT KERAS-------------------------------------------------


    #latest_model="20180927184133.h5"
    latest_model_full_path = models_path +"/"+model_name
    latest_model_full_path_OS = convert_path(latest_model_full_path)

    # load latest model
    classifier = load_model(latest_model_full_path_OS)

    # make predictions
    predicted = classifier.predict(X_test)

    #-----------------------------------------------------------------------------------------------------------
    def compute_class_index(pred_softmax):
        output_list_index = []
        for i in range(pred_softmax.shape[0]):
            max_prediction_index = np.argmax(pred_softmax[i, :])
            output_list_index.append(max_prediction_index)
        return output_list_index
    #-----------------------------------------------------------------------------------------------------------

    predicted_classes = compute_class_index(predicted)

    ts = time.time()
    #timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d%H%M%S')
    #output_csv_file = "testing_results_"+testing_set_out+".csv"
    #output_full_path = forecast_output_path + "/" + output_csv_file
    #output_full_path_OS = convert_path(output_full_path)
    #np.savetxt(output_full_path_OS, predicted_classes, fmt='%.2f', delimiter=',',header='Predicted Queue Time')

    #print('Forecast is complete. Check the following output file for results:\n %s'%output_full_path)


    #-----------------------------------------------------------------------------------------------------------
    def convert_to_class(single_column_array):
        array_of_classes = single_column_array
        i = -1
        for qt in single_column_array:
            i += 1
            if qt == 0:
                array_of_classes[i] = 1 # first class
            elif qt < 5:
                array_of_classes[i] = 2 # second class
            elif qt < 30:
                array_of_classes[i] = 3 # third class
            elif qt < 60:
                array_of_classes[i] = 4 # fourth class
            elif qt < 120:
                array_of_classes[i] = 5 # fifth class
            elif qt < 240:
                array_of_classes[i] = 6 # sixth class
            elif qt < 600:
                array_of_classes[i] = 7 # seventh class
            elif qt > 600:
                array_of_classes[i] = 8 # eighth class
        return array_of_classes
    #-----------------------------------------------------------------------------------------------------------
    test_qt_as_classes=convert_to_class(test_qt)

    #-----------------------------------------------------------------------------------------------------------
    def compute_model_reliability(class_index, target):
        indicator = 0
        for z in range(len(class_index)):
            if class_index[z] == int(target[z]):
                indicator += 1
            model_reliability = indicator/len(class_index)
        return model_reliability
    #-----------------------------------------------------------------------------------------------------------
    modrel=compute_model_reliability(predicted_classes,test_qt_list)

    print("The reliability of the model on the test set was %f"%modrel)

    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(16,6))
    plt.plot(predicted_classes, color="red", label="forecast")
    plt.plot(test_qt_as_classes-1, label="real data")
    plt.legend()
    plt.show()
    
    return modrel, fig
#---------------------------------------------------------------------------------------------------- 
